### 1. Dependencies

In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torch.autograd import Variable
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import DataParallel

import time
import os
import numpy as np
import json
import cv2
from PIL import Image, ImageOps
import random
from tqdm import tqdm
import operator
import itertools
from scipy.io import  loadmat
import logging
from scipy import signal

from utils import data_transforms
from utils import get_paste_kernel, kernel_map
from utils_logging import setup_logger

### 2. Choose between Recasens or GazeNet

- Idea is you can just swap 
models.recasens, dataloader.recasens, training.train_recasens, etc...
- with the following
models.gazenet, dataloader.gazenet, training.train_gazenet

In [2]:
from models.gazenet import GazeNet
from models.__init__ import save_checkpoint, resume_checkpoint
from dataloader.gazenet import GooDataset, GazeDataset
from training.train_gazenet import train, test, GazeOptimizer

In [3]:
# Logger will save the training and test errors to a .log file 
logger = setup_logger(name='first_logger', 
                      log_dir ='./logs/',
                      log_file='random_baseline.log',
                      log_format = '%(asctime)s %(levelname)s %(message)s',
                      verbose=True)

### 3. Dataloaders
- Choose between GazeDataset (Gazefollow dataset) or GooDataset (GooSynth/GooReal)
- Set paths to image directories and pickle paths. For Gazefollow, images_dir and test_images_dir should be the same and both lead to the path containing the train and test folders.

In [4]:
# Dataloaders for GazeFollow
batch_size=32
workers=12
testbatchsize=16

images_dir = '/home/eee198/Documents/datasets/GazeFollowData/'
pickle_path = '/home/eee198/Documents/datasets/GazeFollowData/train_annotations.mat'
test_images_dir = '/home/eee198/Documents/datasets/GazeFollowData/'
test_pickle_path = '/home/eee198/Documents/datasets/GazeFollowData/test_annotations.mat'

train_set = GazeDataset(images_dir, pickle_path, 'train')
train_data_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=workers)

val_set = GazeDataset(test_images_dir, test_pickle_path, 'test')
test_data_loader = torch.utils.data.DataLoader(val_set, batch_size=testbatchsize, num_workers=workers, shuffle=False)

In [4]:
# Dataloaders for GOO
batch_size=32
workers=12
testbatchsize=32

images_dir = '/hdd/HENRI/goosynth/1person/GazeDatasets/'
pickle_path = '/hdd/HENRI/goosynth/picklefiles/trainpickle2to19human.pickle'
test_images_dir = '/hdd/HENRI/goosynth/test/'
test_pickle_path = '/hdd/HENRI/goosynth/picklefiles/testpickle120.pickle'

train_set = GooDataset(images_dir, pickle_path, 'train')
train_data_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=workers)

val_set = GooDataset(test_images_dir, test_pickle_path, 'test')
test_data_loader = torch.utils.data.DataLoader(val_set, batch_size=testbatchsize, num_workers=workers, shuffle=False)

==> Number of Images: 172800
==> Number of Images: 19200


In [5]:
import cv2
from sklearn.metrics import roc_auc_score

def test_random(test_data_loader, logger):
    
    total_loss = []
    total_error = []
    info_list = []
    heatmaps = []

    with torch.no_grad():
        for data in tqdm(test_data_loader, total=len(test_data_loader)):
                     
            image, face_image, gaze_field, eye_position, gt_position, gt_heatmap = \
                data['image'], data['face_image'], data['gaze_field'], data['eye_position'], data['gt_position'], data['gt_heatmap']
            image, face_image, gaze_field, eye_position, gt_position, gt_heatmap = \
                map(lambda x: Variable(x.cuda()), [image, face_image, gaze_field, eye_position, gt_position, gt_heatmap])

            batch_size = image.shape[0]
            
            #direction, predict_heatmap = net([image, face_image, gaze_field, eye_position])
            #predict_heatmap = np.zeros(shape=(batch_size, 56*56))
            #ridx = np.random.randint(low=0, high=56*56, size=(batch_size,))
            #predict_heatmap[:, ridx] = 1.0
            #predict_heatmap = predict_heatmap.reshape((-1, 56,56))
            
            predict_heatmap = np.random.normal(size=(batch_size, 56,56))
            
            final_output = predict_heatmap
            target = gt_position.cpu().data.numpy()
            eye_position = eye_position.cpu().data.numpy()
            predict_heatmap = predict_heatmap

            for f_point, gt_point, eye_point, heatmap in \
                zip(final_output, target, eye_position, predict_heatmap):
                f_point = f_point.reshape([224 // 4, 224 // 4])
                heatmaps.append(f_point)

                h_index, w_index = np.unravel_index(f_point.argmax(), f_point.shape)
                f_point = np.array([w_index / 56., h_index / 56.])

                f_error = f_point - gt_point
                f_dist = np.sqrt(f_error[0] ** 2 + f_error[1] ** 2)

                # angle
                f_direction = f_point - eye_point
                gt_direction = gt_point - eye_point

                norm_f = (f_direction[0] **2 + f_direction[1] ** 2 ) ** 0.5
                norm_gt = (gt_direction[0] **2 + gt_direction[1] ** 2 ) ** 0.5

                f_cos_sim = (f_direction[0]*gt_direction[0] + f_direction[1]*gt_direction[1]) / \
                            (norm_gt * norm_f + 1e-6)
                f_cos_sim = np.maximum(np.minimum(f_cos_sim, 1.0), -1.0)
                f_angle = np.arccos(f_cos_sim) * 180 / np.pi

                #AUC
                heatmap = np.squeeze(heatmap)
                heatmap = cv2.resize(heatmap, (5, 5))
                gt_heatmap = np.zeros((5, 5))
                x, y = list(map(int, gt_point * 5))
                gt_heatmap[y, x] = 1.0
                
                score = roc_auc_score(gt_heatmap.reshape([-1]).astype(np.int32), heatmap.reshape([-1]))

                total_error.append([score, f_dist, f_angle])
                info_list.append(list(f_point))
    info_list = np.array(info_list)

    logger.info('average error: %s'%str(np.mean(np.array(total_error), axis=0)))

    return np.mean(np.array(total_error), axis=0)

In [6]:
test_random(test_data_loader, logger)

100%|██████████| 600/600 [07:54<00:00,  1.26it/s]
average error: [ 0.49728516  0.45361068 76.97099224]


array([ 0.49728516,  0.45361068, 76.97099224])